<a href="https://www.kaggle.com/code/mmellinger66/store-sales-course-base-model?scriptVersionId=105426685" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Store Sales EDA

Finished the [Kaggle Time Series Course](https://www.kaggle.com/learn/time-series). 

I extracted the forecasting model from the course into this notebook.  The LB score is exactly the same.  This notebook is only meant to serve as a succinct reference.

# References

- [Store Sales: Seasonality](https://www.kaggle.com/mmellinger66/store-sales-seasonality)
- https://www.kaggle.com/code/robikscube/time-series-forecasting-with-machine-learning-yt
- https://www.kaggle.com/code/robikscube/pt2-time-series-forecasting-with-xgboost

# Next Steps

## EDAs

- [First kaggle notebook. Following TS tutorial](https://www.kaggle.com/howoojang/first-kaggle-notebook-following-ts-tutorial)

## Models

- [Simple LB 0.43911 (Lasso, Ridge, NN)](https://www.kaggle.com/avtobusbratiev/simple-lb-0-43911-lasso-ridge-nn)
- [Simple TS + Ridge](https://www.kaggle.com/dkomyagin/simple-ts-ridge)

# Load Libraries

In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

In [2]:
comp_dir = Path('../input/store-sales-time-series-forecasting')

# Load Train/Test Store Sales

In [3]:
store_sales = pd.read_csv(
    comp_dir / 'train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()


df_test = pd.read_csv(
    comp_dir / 'test.csv',
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
df_test['date'] = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

In [4]:
store_sales

sales
store_nbr family     date                 
1         AUTOMOTIVE 2013-01-01   0.000000
                     2013-01-02   2.000000
                     2013-01-03   3.000000
                     2013-01-04   3.000000
                     2013-01-05   5.000000
...                                    ...
9         SEAFOOD    2017-08-11  23.830999
                     2017-08-12  16.859001
                     2017-08-13  20.000000
                     2017-08-14  17.000000
                     2017-08-15  16.000000

[3000888 rows x 1 columns]

In [5]:
# store_sales.date

# Target

In [6]:
y = store_sales.unstack(['store_nbr', 'family']).loc["2017"]
y

sales                                                \
store_nbr           1                                                 
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2017-01-01        0.0       0.0    0.0       0.0   0.0     0.000000   
2017-01-02        5.0       0.0    0.0    1434.0   0.0   166.819000   
2017-01-03        4.0       0.0    4.0    3081.0   2.0   519.348022   
2017-01-04        1.0       0.0    4.0    3039.0   2.0   543.250977   
2017-01-05        2.0       0.0    3.0    2617.0   0.0   533.479980   
...               ...       ...    ...       ...   ...          ...   
2017-08-11        1.0       0.0    1.0    1006.0   0.0   145.606995   
2017-08-12        6.0       0.0    3.0    1659.0   0.0   243.220001   
2017-08-13        1.0       0.0    1.0     803.0   0.0   136.679001   
2017-08-14        1.0       0.0    6.0    2201.0   0.0   346.037994   
2017-08-15        4.0       0.0    4.0    1942.0   0.0   329.541016   

                                                     ...            \
store_nbr                                            ...         9   
family     CELEBRATION CLEANING   DAIRY        DELI  ... MAGAZINES   
date                                                 ...             
2017-01-01         0.0      0.0     0.0    0.000000  ...       0.0   
2017-01-02         0.0    332.0   376.0   44.980000  ...       5.0   
2017-01-03        15.0    952.0  1045.0  209.300003  ...       2.0   
2017-01-04        17.0   1055.0  1029.0  135.944000  ...       3.0   
2017-01-05        40.0    918.0   853.0  137.005997  ...       2.0   
...                ...      ...     ...         ...  ...       ...   
2017-08-11         4.0    341.0   343.0   64.302002  ...       5.0   
2017-08-12         3.0    351.0   526.0   99.487999  ...       2.0   
2017-08-13         1.0    169.0   266.0   47.770000  ...       3.0   
2017-08-14         4.0    571.0   699.0  154.578003  ...      12.0   
2017-08-15        21.0    703.0   602.0  116.402000  ...      11.0   

                                                                           \
store_nbr                                                                   
family           MEATS PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS   
date                                                                        
2017-01-01    0.000000           0.0          0.0                     0.0   
2017-01-02  659.570007        1243.0         11.0                    41.0   
2017-01-03  547.364014         876.0          6.0                    15.0   
2017-01-04  395.287994         677.0          6.0                    13.0   
2017-01-05  470.768005         604.0          7.0                    10.0   
...                ...           ...          ...                     ...   
2017-08-11  309.244995         373.0         11.0                     2.0   
2017-08-12  260.298004         400.0          7.0                    10.0   
2017-08-13  327.205994         510.0          2.0                     9.0   
2017-08-14  330.975006         445.0          2.0                    14.0   
2017-08-15  449.227997         522.0          6.0                     6.0   

                                                                               \
store_nbr                                                                       
family         POULTRY PREPARED FOODS      PRODUCE SCHOOL AND OFFICE SUPPLIES   
date                                                                            
2017-01-01    0.000000       0.000000     0.000000                        0.0   
2017-01-02  843.596008     115.188995  3136.895996                        1.0   
2017-01-03  714.659973     133.039001  3229.558105                        1.0   
2017-01-04  536.830017      75.201004  1491.416992                        7.0   
2017-01-05  414.100006     113.698997  1566.821045                        1.0   
...           

# Create training data

In [7]:
fourier = CalendarFourier(freq='M', order=4)
dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=1,
    seasonal=True,
    additional_terms=[fourier],
    drop=True,
)
X = dp.in_sample()
X['NewYear'] = (X.index.dayofyear == 1)

In [8]:
X

,const,trend,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)","sin(1,freq=M)","cos(1,freq=M)","sin(2,freq=M)","cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)","sin(4,freq=M)","cos(4,freq=M)",NewYear
date,,,,,,,,,,,,,,,,,
2017-01-01,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,True
2017-01-02,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.201299,0.979530,0.394356,0.918958,0.571268,0.820763,0.724793,0.688967,False
2017-01-03,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.394356,0.918958,0.724793,0.688967,0.937752,0.347305,0.998717,-0.050649,False
2017-01-04,1.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.571268,0.820763,0.937752,0.347305,0.968077,-0.250653,0.651372,-0.758758,False
2017-01-05,1.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.724793,0.688967,0.998717,-0.050649,0.651372,-0.758758,-0.101168,-0.994869,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-11,1.0,223.0,0.0,0.0,0.0,0.0,1.0,0.0,0.897805,-0.440394,-0.790776,-0.612106,-0.201299,0.979530,0.968077,-0.250653,False
2017-08-12,1.0,224.0,0.0,0.0,0.0,0.0,0.0,1.0,0.790776,-0.612106,-0.968077,-0.250653,0.394356,0.918958,0.485302,-0.874347,False
2017-08-13,1.0,225.0,0.0,0.0,0.0,0.0,0.0,0.0,0.651372,-0.758758,-0.988468,0.151428,0.848644,0.528964,-0.299363,-0.954139,False


# Linear Regression Model

In [9]:
# Simply using Ridge will improve the score

model = Ridge(fit_intercept=True, solver='auto', alpha=0.5, normalize=True)
# model = LinearRegression(fit_intercept=False)
model.fit(X, y)
y_pred = pd.DataFrame(model.predict(X), index=X.index, columns=y.columns)

In [10]:
y_pred.head()

sales                                                          \
store_nbr           1                                                           
family     AUTOMOTIVE BABY CARE    BEAUTY    BEVERAGES     BOOKS BREAD/BAKERY   
date                                                                            
2017-01-01   1.147156       0.0  1.006750   650.711114  0.207317   113.487636   
2017-01-02   4.012898       0.0  3.466052  2296.270421  0.573187   401.896876   
2017-01-03   3.933510       0.0  3.537022  2376.162414  0.804131   396.600131   
2017-01-04   3.969894       0.0  3.408894  2477.704000  0.863402   444.841239   
2017-01-05   3.829655       0.0  3.026328  2195.331531  0.660689   401.325145   

                                                            ...            \
store_nbr                                                   ...         9   
family     CELEBRATION    CLEANING       DAIRY        DELI  ... MAGAZINES   
date                                                        ...             
2017-01-01    3.794296  199.501107  227.196277   41.783909  ...  1.411856   
2017-01-02   12.908956  690.676207  781.062022  147.193476  ...  3.447104   
2017-01-03   14.420038  779.965203  784.468982  143.758748  ...  2.970190   
2017-01-04   14.587128  844.657932  876.972799  145.800692  ...  2.644287   
2017-01-05   16.791008  715.375500  759.096377  126.264347  ...  2.438306   

                                                                           \
store_nbr                                                                   
family           MEATS PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS   
date                                                                        
2017-01-01  162.398834    233.412868     3.245875                5.968915   
2017-01-02  478.056978    700.676731     9.013801               16.997233   
2017-01-03  454.960729    671.651455     8.149008               16.143299   
2017-01-04  420.995718    596.617898     8.475171               14.668484   
2017-01-05  537.615568    548.180236     6.743227               13.853948   

                                                                               \
store_nbr                                                                       
family         POULTRY PREPARED FOODS      PRODUCE SCHOOL AND OFFICE SUPPLIES   
date                                                                            
2017-01-01  190.850655      48.762323   685.072869                  -0.642251   
2017-01-02  544.459826     130.303397  1911.404746                   0.468305   
2017-01-03  518.954810     128.537361  2461.255835                   5.439751   
2017-01-04  476.229153     120.624991  1656.314191                   1.088597   
2017-01-05  438.639082     118.154383  1599.145314                   1.984544   

                       
store_nbr              
family        SEAFOOD  
date                   
2017-01-01   6.851256  
2017-01-02  18.593730  
2017-01-03  18.248528  
2017-01-04  16.580535  
2017-01-05  15.351088  

[5 rows x 1782 columns]

# Create features for test set

In [11]:
X_test = dp.out_of_sample(steps=16)
X_test.index.name = 'date'
X_test['NewYear'] = (X_test.index.dayofyear == 1)

# Submission File

In [12]:
y_submit = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)
y_submit = y_submit.stack(['store_nbr', 'family'])
y_submit = y_submit.join(df_test.id).reindex(columns=['id', 'sales'])
y_submit.to_csv('submission.csv', index=False)
y_submit

id        sales
date       store_nbr family                                          
2017-08-16 1         AUTOMOTIVE                  3000888     4.047986
                     BABY CARE                   3000889     0.000000
                     BEAUTY                      3000890     3.334732
                     BEVERAGES                   3000891  2265.860059
                     BOOKS                       3000892     0.448142
...                                                  ...          ...
2017-08-31 9         POULTRY                     3029395   404.219418
                     PREPARED FOODS              3029396   114.523504
                     PRODUCE                     3029397  1445.600089
                     SCHOOL AND OFFICE SUPPLIES  3029398    34.897744
                     SEAFOOD                     3029399    18.739132

[28512 rows x 2 columns]